<a href="https://colab.research.google.com/github/HenryWatson11/WaveDetectorInator3000/blob/main/WaveDetetorModelMK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib
import keras
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import random
import os
from matplotlib import pyplot as plt



In [ ]:
!pip install piexif

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.listdir("./drive/MyDrive/Wave Detector Images")

Mounted at /content/drive


['Test', 'Train']

In [ ]:
from keras.layers import Dense, Activation, Flatten

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Data augmentation and normalization
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory('./drive/MyDrive/Wave Detector Images/Train', target_size=(224, 224), batch_size=32, class_mode='categorical')

# Build CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10)

# Evaluate on the test set
test_generator = test_datagen.flow_from_directory('./drive/MyDrive/Wave Detector Images/Test', target_size=(224, 224), batch_size=32, class_mode='categorical')
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 94 images belonging to 5 classes.
Epoch 1/10
3/3 [==============================] - 12s 3s/step - loss: 34.7688 - accuracy: 0.2766
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 28.5991 - accuracy: 0.2979
Epoch 3/10
3/3 [==============================] - 8s 3s/step - loss: 20.3577 - accuracy: 0.4787
Epoch 4/10
3/3 [==============================] - 6s 2s/step - loss: 19.4878 - accuracy: 0.3404
Epoch 5/10
3/3 [==============================] - 8s 2s/step - loss: 7.2069 - accuracy: 0.2979
Epoch 6/10
3/3 [==============================] - 6s 2s/step - loss: 4.6508 - accuracy: 0.4787
Epoch 7/10
3/3 [==============================] - 9s 2s/step - loss: 2.1810 - accuracy: 0.5213
Epoch 8/10
3/3 [==============================] - 6s 2s/step - loss: 1.6708 - accuracy: 0.4043
Epoch 9/10
3/3 [==============================] - 6s 2s/step - loss: 1.3681 - accuracy: 0.4787
Epoch 10/10
3/3 [==============================] - 6s 2s/step - loss: 1.0441 - accuracy: 0.6489
Foun

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump
import os

# Function to extract features using VGG16 model
def extract_features(directory):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    features = dict()

    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        try:
            #print(f"Processing image: {filename}")

            image = load_img(path, target_size=(224, 224))
            image = img_to_array(image)
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            image_id = filename.split('.')[0]
            features[image_id] = feature
        except Exception as e:
            print(f"Error processing image {filename}: {str(e)}")

    return features

# Extract features for the training dataset
train_features1 = extract_features('./drive/MyDrive/Wave Detector Images/Train/1')
train_features3 = extract_features('./drive/MyDrive/Wave Detector Images/Train/3')
train_features5 = extract_features('./drive/MyDrive/Wave Detector Images/Train/5')
train_features7 = extract_features('./drive/MyDrive/Wave Detector Images/Train/7')
train_features9 = extract_features('./drive/MyDrive/Wave Detector Images/Train/9')


In [ ]:
test_features1 = extract_features('./drive/MyDrive/Wave Detector Images/Test/1')
test_features3 = extract_features('./drive/MyDrive/Wave Detector Images/Test/3')
test_features5 = extract_features('./drive/MyDrive/Wave Detector Images/Test/5')
test_features7 = extract_features('./drive/MyDrive/Wave Detector Images/Test/7')

In [87]:
X_train = []
y_train = []
x = 1
grand_features = [train_features1, train_features3, train_features5, train_features7]
for gf in grand_features:
  for features in gf.values():
    X_train.append(features)
    y_train.append(x)
  x = x + 2
X_train = np.array(X_train)
X_train = X_train[0]
y_train = np.array(y_train)
print(X_train.shape)

(1, 4096)


In [ ]:
X_test = []
y_test = []
grand_features_test = [test_features1, test_features3, test_features5, test_features7]
for gf in grand_features:
  for features in gf:
    X_test.append(features)
    y_test.append(x)
  x = x + 2
X_test = np.array(X_test)
y_test = np.array(y_test)

In [62]:
from sklearn.linear_model import LogisticRegression

In [88]:

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
predictions = lr_model.predict(X_test)
sum(predictions == y_test) / len(y_test)

ValueError: ignored